## ICSI corpus. 
Go through the list of files and extract passage and the summaries, using the DialogueActs as intermediary

In [15]:
import glob, os
#import xml.etree.ElementTree as et
from lxml import etree as et
from collections import OrderedDict, defaultdict
ROOT_DIR = "/home/rajivn/W266/W266-fall-2020-hwu-rnair/data/ICSI_plus_NXT/ICSIplus"

### Extract words

In [16]:
meeting_dict = dict() # key = meeting, value = ordered dict of word id: word
cur_word = None
for word_f in sorted(glob.glob(ROOT_DIR+"/Words/*xml")):
    m = word_f.split("/")[-1].split(".")
    meeting_name = m[0]+"."+m[1]
    root = et.parse(word_f).getroot()
    nodes = root.findall('*')
    meet_words = OrderedDict()

    for node in nodes:
        if node.tag=='w':
            meet_words[node.attrib['{http://nite.sourceforge.net/}id']] = node.text
            cur_word = node.text
        for w in node.values():
            if "disfmarker" in w:
                meet_words [node.attrib['{http://nite.sourceforge.net/}id']] = cur_word
    meeting_dict[meeting_name] = meet_words
    
#meeting_dict['Bdb001.D']['Bdb001.w.2,391']

##### Extract complete text and store in files under ../ICSI_plus_NXT/processing/_meeting_id_words.txt file

In [17]:
len(meeting_dict.keys())

for meeting, words in meeting_dict.items():
    with open("../data/ICSI_plus_NXT/processing/"+meeting+"_words.txt", "w") as f:
        for word in words.values():
            f.write(word)
            f.write(" ")

### Extract dialogues

Dialogues are equivalent to complete sentences

In [18]:
meeting_dialogues = dict() # key = meeting name, value = list of tuples start, end word ids(end can be same as start)
for dialog_f in sorted(glob.glob(ROOT_DIR+"/DialogueActs/*acts.xml")):
    m = dialog_f.split("/")[-1].split(".")
    meeting_name = m[0]+"."+m[1]
    root = et.parse(dialog_f).getroot()
    dlist = root.findall('dialogueact')
    for dl in dlist:
        nodes = dl.findall("{http://nite.sourceforge.net/}child")
        dialog_id = dl.attrib['{http://nite.sourceforge.net/}id']

        for dl_c in nodes:
            words = dl_c.attrib['href']
            words = words.split("#")[1]
            try:
                start, end = words.split("..")
                start, end = start.replace("id", ""), end.replace("id", "")
                start, end = start.replace("(", ""), end.replace("(", "")
                start, end = start.replace(")", ""), end.replace(")", "")

    #            print(start, end)
            except:
                start = end = words.replace("id", "").replace("(", "").replace(")", "")
    #            print(start)
            if meeting_name not in meeting_dialogues:
                meeting_dialogues[meeting_name] = OrderedDict()
            meeting_dialogues[meeting_name][dialog_id] = ((start, end))




#####  Extract complete dialogues and store in files under ../ICSI_plus_NXT/processing/_meeting_id_dialogues.txt file

In [19]:
print(len(meeting_dialogues.keys()))
#print(meeting_dialogues['Bdb001'])
dialogues = dict() # key = dialogue id, value = text (required for summaries)
for meeting, sentence_map in meeting_dialogues.items():
    with open("../data/ICSI_plus_NXT/processing/"+meeting+"_dialogues.txt", "w") as f:
        starts, ends = dict(), dict()
        for d_id, (start, end) in sentence_map.items():
            if d_id == 'Bdb001.D.dialogueact285':
                print ("here")
            starts[start] = d_id
            ends[end] = d_id
        #go through entire text and start printing if start token is in "starts". stop printing if end token is in "ends"
        print_w = False
        cur_d = None
        for word_id, word in meeting_dict[meeting].items():
            if word_id == 'Bdb001.w.2,391':
                print("here 2")
            if word_id in starts:
                cur_d = starts[word_id]
                print_w = True
            if print_w:
                f.write(word + " ")
                dialogues[cur_d] = dialogues.get(cur_d, "") + word+ " "
            if word_id in ends:
                print_w = False
                f.write("\n")



465
here
here 2


In [20]:
print(dialogues['Bdb001.D.dialogueact285'])

P - file for each frame is storing a vector of cepstral or PLP values , 


### Extract summaries

In [21]:
meeting_summaries = defaultdict(list) # key = meeting name, value = list of tuples start, end word ids(end can be same as start)

for dialog_f in sorted(glob.glob(ROOT_DIR+"/Contributions/Summarization/extractive/*extsumm.xml")):
    m = dialog_f.split("/")[-1].split(".")
    meeting_name = m[0]+"."+m[1]
    root = et.parse(dialog_f).getroot()
    dlist = root.findall('extsumm/{http://nite.sourceforge.net/}child')
    for dl in dlist:
        words = dl.attrib['href']
        words = words.split("#")[1]
        try:
            start, end = words.split("..")
            start, end = start.replace("id", ""), end.replace("id", "")
            start, end = start.replace("(", ""), end.replace("(", "")
            start, end = start.replace(")", ""), end.replace(")", "")
            
#            print(start, end)
        except:
            start = end = words.replace("id", "").replace("(", "").replace(")", "")
#            print(start)
        meeting_summaries[meeting_name].append((start, end))


In [22]:
for meeting, summ_list in meeting_summaries.items():
    with open("../data/ICSI_plus_NXT/processing/"+meeting+"_summaries.txt", "w") as f:
        for start, end in summ_list:
            st = int(start.split('dialogueact')[1])
            en = int(start.split('dialogueact')[1])+1
            for d in range(st, en):
                key = start.split('dialogueact')[0]+"dialogueact"+str(d)
                if key not in dialogues:
                    print("missing", key)
                f.write(dialogues.get(key))
                f.write("\n")
